<a href="https://colab.research.google.com/github/Tony14100/portfolio/blob/main/gradio_%D0%B4%D0%BB%D1%8F_%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B0_%D0%B0%D0%BA%D1%86%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект: Прогнозирование цен акций с помощью машинного обучения и Gradio

## Описание проекта
Этот проект демонстрирует простой подход к прогнозированию цен акций с использованием исторических данных и модели линейной регрессии. Проект включает предобработку данных, обучение отдельной модели для каждой акции и создание интерактивного веб-приложения с использованием библиотеки Gradio для получения прогнозов.

## Цель проекта
Основная цель проекта — создать инструмент, который позволяет пользователям выбирать акцию из списка и получать прогнозируемую цену на указанную будущую дату на основе обученных моделей. Этот проект может служить основой для более сложных систем прогнозирования и является хорошим примером создания интерактивных приложений машинного обучения.

## Используемые данные
В проекте используются данные о ценах акций, загруженные из набора данных с Kaggle: "Crypto and Stock Market Data for Financial Analysis". Данные содержат временные метки, названия акций и цены закрытия.

## Шаги проекта
1.  **Загрузка данных**: Данные о ценах акций загружаются из CSV файла.
2.  **Предобработка данных**: Выбираются необходимые столбцы ('timestamp', 'name', 'last'), столбец с ценой преобразуется в числовой формат, а столбец с временной меткой — в формат datetime. Данные сортируются по названию акции и временной метке.
3.  **Обучение моделей**: Для каждой уникальной акции в наборе данных обучается отдельная модель линейной регрессии. Модели сохраняются в словаре для последующего использования.
4.  **Функция прогнозирования**: Создается функция, которая принимает название акции и будущую дату, выбирает соответствующую обученную модель и прогнозирует цену на указанную дату.
5.  **Интерфейс Gradio**: Создается интерактивный пользовательский интерфейс с использованием библиотеки Gradio. Интерфейс включает выпадающий список для выбора акции и поле ввода для указания даты прогноза.
6.  **Интеграция**: Функция прогнозирования интегрируется с интерфейсом Gradio, позволяя пользователям взаимодействовать с моделью через веб-интерфейс.

## Как пользоваться приложением
После запуска ячейки с кодом Gradio, появится интерактивный интерфейс. Выберите акцию из выпадающего списка, введите дату в формате ГГГГ-ММ-ДД в соответствующее поле и нажмите кнопку для получения прогноза цены на указанную дату.

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adrianjuliusaluoch/crypto-and-stock-market-data-for-financial-analysis")

print("Path to dataset files:", path)

100%|██████████| 2.51M/2.51M [00:00<00:00, 3.09MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/adrianjuliusaluoch/crypto-and-stock-market-data-for-financial-analysis/versions/25


In [2]:
import os

# The 'path' variable was defined in the previous cell
print(os.listdir(path))

['cryptocurrency.csv', 'stocks.csv']


In [5]:
# Посмотрим
import pandas as pd

# The 'path' variable was defined in the previous cell and contains the directory
file_path = os.path.join(path, 'cryptocurrency.csv')
df = pd.read_csv(file_path)

# Display the first few rows to verify
display(df.head())

,timestamp,name,symbol,price_usd,vol_24h,total_vol,chg_24h,chg_7d,market_cap
0,2025-09-14 02:34:11,Tether USDt,USDT,1.0004,$120.02B,82.15%,+0.03%,+0.01%,$170.07B
1,2025-09-14 02:34:11,XRP,XRP,3.1206,$5.61B,3.90%,+0.60%,+11.36%,$186.41B
2,2025-09-14 02:34:11,Solana,SOL,242.07,$7.56B,5.81%,+0.92%,+21.11%,$131.80B
3,2025-09-14 02:34:11,Dogecoin,DOGE,0.289093,$9.02B,4.15%,+5.30%,+34.11%,$43.79B
4,2025-09-14 02:34:11,Bitcoin,BTC,"115,897.70",$34.88B,27.85%,-0.11%,+5.20%,$2.31T


In [6]:
import os
import pandas as pd

# The 'path' variable was defined in a previous cell and contains the directory
stock_file_path = os.path.join(path, 'stocks.csv')
df_stocks = pd.read_csv(stock_file_path)

# Display the first few rows to verify
display(df_stocks.head())

,timestamp,name,last,high,low,chg_,chg_%,vol_,time
0,2025-09-14 01:46:01,Boeing,215.94,220.00,215.70,-4.05,-1.84%,11.09M,12/09
1,2025-09-14 01:46:01,Chevron,157.11,159.78,157.10,-1.14,-0.72%,5.55M,12/09
2,2025-09-14 01:46:01,Citigroup,99.44,99.70,98.44,0.62,+0.63%,9.5M,12/09
3,2025-09-14 01:46:01,Caterpillar,431.52,432.61,427.75,0.14,+0.03%,2.34M,12/09
4,2025-09-14 01:46:01,Microsoft,509.90,512.55,503.85,8.89,+1.77%,23.62M,12/09


In [9]:
# Выбираем необходимые столбцы
df_stocks_processed = df_stocks[['timestamp', 'name', 'last']].copy()

# Преобразуем столбец 'last' в числовой формат, удаляя символы '$' и ','
df_stocks_processed['last'] = df_stocks_processed['last'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df_stocks_processed['last'] = pd.to_numeric(df_stocks_processed['last'])

# Преобразуем столбец 'timestamp' в формат datetime
df_stocks_processed['timestamp'] = pd.to_datetime(df_stocks_processed['timestamp'])

# Сортируем данные по названию акции и дате
df_stocks_processed = df_stocks_processed.sort_values(by=['name', 'timestamp'])

# Выводим первые строки обработанных данных
display(df_stocks_processed.head())

,timestamp,name,last
131062,2025-03-17 00:45:59,3M,150.41
131032,2025-03-17 01:46:00,3M,150.41
131002,2025-03-17 02:46:01,3M,150.41
130972,2025-03-17 03:45:59,3M,150.41
130942,2025-03-17 04:46:00,3M,150.41


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

# Для простоты примера, возьмем только одну акцию для обучения модели
# В реальном приложении нужно будет обучать модель для каждой акции или использовать более сложный подход
# Выберем, например, акцию "Apple"
df_apple = df_stocks_processed[df_stocks_processed['name'] == 'Apple'].copy()

# Преобразуем дату в числовой формат для модели
df_apple['timestamp_numeric'] = (df_apple['timestamp'] - df_apple['timestamp'].min()).dt.total_seconds()

# Подготовка данных для обучения
X = df_apple[['timestamp_numeric']]
y = df_apple['last']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация и обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

print("Модель обучена.")

Модель обучена.


In [15]:
from datetime import timedelta

# Используем обученные модели для всех акций
def predict_stock_price(stock_name, future_date):
    """
    Прогнозирует цену акции на заданную дату, используя соответствующую модель.
    """
    if stock_name not in stock_models:
        return f"Модель для акции '{stock_name}' не найдена."

    model = stock_models[stock_name]

    # Найдем минимальную временную метку для этой конкретной акции
    # Это важно, так как timestamp_numeric был рассчитан относительно начала данных для каждой акции
    df_stock = df_stocks_processed[df_stocks_processed['name'] == stock_name].copy()
    min_timestamp = df_stock['timestamp'].min()

    # Преобразуем будущую дату в числовой формат относительно начала данных для этой акции
    if not isinstance(future_date, pd.Timestamp):
        future_date = pd.to_datetime(future_date)

    time_difference = (future_date - min_timestamp).total_seconds()

    # Прогнозируем цену с помощью обученной модели
    predicted_price = model.predict([[time_difference]])

    return f"Прогнозируемая цена для {stock_name} на {future_date.strftime('%Y-%m-%d')} составляет: {predicted_price[0]:.2f}"

# Пример использования функции
future_date_example = pd.to_datetime('2025-10-01') # Пример будущей даты
print(predict_stock_price('Apple', future_date_example))
print(predict_stock_price('Google', future_date_example)) # Пример для другой акции

Прогнозируемая цена для Apple на 2025-10-01 составляет: 226.45
Модель для акции 'Google' не найдена.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [12]:
!pip install -q gradio

In [16]:
import gradio as gr
from datetime import datetime
import pandas as pd # Убедимся, что pandas импортирован

def predict_stock(stock_name, future_date_str):
    """
    Функция-обертка для predict_stock_price для использования в Gradio.
    Преобразует строку даты в объект datetime.
    """
    try:
        future_date = pd.to_datetime(future_date_str)
        return predict_stock_price(stock_name, future_date)
    except ValueError:
        return "Неверный формат даты. Используйте формат ГГГГ-ММ-ДД."
    except Exception as e:
        return f"Произошла ошибка: {e}"

# Получаем список доступных акций из словаря обученных моделей
available_stocks = list(stock_models.keys())

# Создаем интерфейс Gradio
interface = gr.Interface(
    fn=predict_stock,
    inputs=[
        gr.Dropdown(choices=available_stocks, label="Выберите акцию"), # Используем выпадающий список
        gr.Textbox(label="Дата для прогноза (ГГГГ-ММ-ДД)")
        ],
    outputs="text",
    title="Прогноз цены акций",
    description="Выберите акцию из списка и введите дату для получения прогноза цены."
)

# Запускаем интерфейс
interface.launch(inline=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf4ca4342360628b19.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [14]:
# Обучаем модель для каждой уникальной акции
unique_stocks = df_stocks_processed['name'].unique()
stock_models = {}

for stock_name in unique_stocks:
    df_stock = df_stocks_processed[df_stocks_processed['name'] == stock_name].copy()

    if len(df_stock) > 1: # Убедимся, что данных для обучения достаточно
        # Преобразуем дату в числовой формат для модели
        df_stock['timestamp_numeric'] = (df_stock['timestamp'] - df_stock['timestamp'].min()).dt.total_seconds()

        # Подготовка данных для обучения
        X = df_stock[['timestamp_numeric']]
        y = df_stock['last']

        # Инициализация и обучение модели линейной регрессии
        model = LinearRegression()
        model.fit(X, y) # Обучаем на всех доступных данных для этой акции

        stock_models[stock_name] = model
    else:
        print(f"Недостаточно данных для обучения модели для акции: {stock_name}")

print(f"Модели обучены для {len(stock_models)} акций.")

Модели обучены для 31 акций.
